In [48]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from sklearn.feature_selection import SelectFromModel, RFE

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import LinearSVC, LinearSVR

from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

pd.set_option('display.max_columns', None)

In [2]:
data = pd.read_csv('database/clean.csv', index_col=0)
# data.dropna(inplace=True)

data.head()

,MSSubClass,MSZoning,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,60,RL,8450,Pave,None,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,6.753438,GasA,Ex,Y,SBrkr,854,0,7.444833,1,0,2,1,3,1,Gd,Typ,0,None,Attchd,2003.0,RFn,2,TA,TA,Y,0,61,0,0,0,0,None,None,0,2,2008,WD,Normal,12.247699
2,20,RL,9600,Pave,None,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,7.141245,GasA,Ex,Y,SBrkr,0,0,7.141245,0,1,2,0,3,1,TA,Typ,1,TA,Attchd,1976.0,RFn,2,TA,TA,Y,298,0,0,0,0,0,None,None,0,5,2007,WD,Normal,12.109016
3,60,RL,11250,Pave,None,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,6.825460,GasA,Ex,Y,SBrkr,866,0,7.488294,1,0,2,1,3,1,Gd,Typ,1,TA,Attchd,2001.0,RFn,2,TA,TA,Y,0,42,0,0,0,0,None,None,0,9,2008,WD,Normal,12.317171
4,70,RL,9550,Pave,None,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,6.629363,GasA,Gd,Y,SBrkr,756,0,7.448916,1,0,1,0,3,1,Gd,Typ,1,Gd,Detchd,1998.0,Unf,3,TA,TA,Y,0,35,272,0,0,0,None,None,0,2,2006,WD,Abnorml,11.849405
5,60,RL,14260,Pave,None,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,7.044033,GasA,Ex,Y,SBrkr,1053,0,7.695758,1,0,2,1,4,1,Gd,Typ,1,TA,Attchd,2000.0,RFn,3,TA,TA,Y,192,84,0,0,0,0,None,None,0,12,2008,WD,Normal,12.429220


In [4]:
cat_features = [f for f in data.columns if data[f].dtype=='O']

label = LabelEncoder()

for f in cat_features:
    data[f] = label.fit_transform(data[f])

In [5]:
data.head()

,MSSubClass,MSZoning,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,60,3,8450,1,1,3,3,0,4,0,5,2,2,0,5,7,5,2003,2003,1,0,12,13,2,4,2,2,4,3,2,706,6,0,150,6.753438,1,0,1,4,854,0,7.444833,1,0,2,1,3,1,2,6,0,3,1,89,2,2,5,5,2,0,61,0,0,0,0,3,4,0,2,2008,8,4,12.247699
2,20,3,9600,1,1,3,3,0,2,0,24,1,2,0,2,6,8,1976,1976,1,0,8,8,3,4,1,2,4,1,0,978,6,0,284,7.141245,1,0,1,4,0,0,7.141245,0,1,2,0,3,1,3,6,1,5,1,62,2,2,5,5,2,298,0,0,0,0,0,3,4,0,5,2007,8,4,12.109016
3,60,3,11250,1,1,0,3,0,4,0,5,2,2,0,5,7,5,2001,2002,1,0,12,13,2,4,2,2,4,2,2,486,6,0,434,6.825460,1,0,1,4,866,0,7.488294,1,0,2,1,3,1,2,6,1,5,1,87,2,2,5,5,2,0,42,0,0,0,0,3,4,0,9,2008,8,4,12.317171
4,70,3,9550,1,1,0,3,0,0,0,6,2,2,0,5,7,5,1915,1970,1,0,13,15,3,4,0,4,1,3,0,216,6,0,540,6.629363,1,2,1,4,756,0,7.448916,1,0,1,0,3,1,2,6,1,2,5,84,3,3,5,5,2,0,35,272,0,0,0,3,4,0,2,2006,8,0,11.849405
5,60,3,14260,1,1,0,3,0,2,0,15,2,2,0,5,8,5,2000,2000,1,0,12,13,2,4,2,2,4,0,2,655,6,0,490,7.044033,1,0,1,4,1053,0,7.695758,1,0,2,1,4,1,2,6,1,5,1,86,2,3,5,5,2,192,84,0,0,0,0,3,4,0,12,2008,8,4,12.429220


# Feature Selection

In [195]:
FEATURES                   = data.drop(columns=['SalePrice'])
TARGET                     = data['SalePrice']
NUMBER_OF_FEATURES_TO_KEEP = 20

X_train, X_test, y_train, y_test = train_test_split(FEATURES, TARGET, test_size=0.1, random_state=10)

# fs = SelectFromModel(Lasso(alpha=0.001), max_features=NUMBER_OF_FEATURES_TO_KEEP)
fs = RFE(estimator=RandomForestRegressor(), n_features_to_select=NUMBER_OF_FEATURES_TO_KEEP, step=10, verbose=5)

fs.fit(X_train, y_train)

features_selected = X_train.columns[fs.get_support()]
features_selected

Fitting estimator with 72 features.
Fitting estimator with 62 features.
Fitting estimator with 52 features.
Fitting estimator with 42 features.
Fitting estimator with 32 features.
Fitting estimator with 22 features.


Index(['MSZoning', 'LotArea', 'Neighborhood', 'OverallQual', 'OverallCond',
       'YearBuilt', 'YearRemodAdd', 'BsmtQual', 'BsmtFinSF1', 'BsmtUnfSF',
       'TotalBsmtSF', 'CentralAir', '2ndFlrSF', 'GrLivArea', 'GarageType',
       'GarageYrBlt', 'GarageCars', 'WoodDeckSF', 'OpenPorchSF', 'MoSold'],
      dtype='object')

# Splitting

In [66]:
top_correlated_features = ['OverallQual', 'GrLivArea', 'GarageCars', 'TotalBsmtSF', 'FullBath', 'YearBuilt', 'YearRemodAdd']

FEATURES = data.drop(columns=['SalePrice'])
# FEATURES = data[top_correlated_features]
# FEATURES = data[features_selected]

TARGET   = data['SalePrice']

X_train, X_test, y_train, y_test = train_test_split(FEATURES, TARGET, test_size=0.2, random_state=10)

In [68]:
y_test

Id
1389    12.841328
1432    11.875838
721     12.524530
578     12.010672
36      12.641100
          ...    
697     11.396403
800     12.072547
455     12.144203
1110    12.542548
1136    11.532738
Name: SalePrice, Length: 292, dtype: float64

# MODELS

In [8]:
def kaggle_score(y_true, y_pred):
    y_true = np.log(y_true)
    y_pred = np.log(y_pred)
    return np.sqrt(mean_squared_error(y_true, y_pred))

In [69]:
rfr = RandomForestRegressor(random_state=10)
rfr.fit(X_train, y_train)

rfr_pred = rfr.predict(X_test)

# kaggle_score(y_test, rfr_pred)

In [70]:
xgb = GradientBoostingRegressor(learning_rate=0.2, n_estimators=100, verbose=0, random_state=10)
xgb.fit(X_train, y_train)

xgb_pred = xgb.predict(X_test)

# kaggle_score(y_test, xgb_pred)

In [71]:
svc = LinearSVR(random_state=10, verbose=1)
svc.fit(X_train, y_train)

svc_pred = svc.predict(X_test)

# kaggle_score(y_test, xgb_pred)

[LibLinear]

/home/atom/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


# Evaluation

In [188]:
def kaggle_score(y_true, y_pred):
    y_true = np.log(y_true)
    y_pred = np.log(y_pred)
    return np.sqrt(mean_squared_error(y_true, y_pred))


In [73]:
rfr_pred = np.exp(rfr_pred)-1
xgb_pred = np.exp(xgb_pred)-1
svc_pred = np.exp(svc_pred)-1

y_test  = np.exp(y_test)-1

In [74]:
kaggle_score(y_test, rfr_pred)

0.14193865064195557

In [75]:
kaggle_score(y_test, xgb_pred)

0.13965804133929147

In [77]:
kaggle_score(y_test, svc_pred)

0.2934320979800162

# TEST SET

In [11]:
test = pd.read_csv('database/test.csv', index_col=0)
test.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,Norm,1Fam,1Story,5,6,1961,1961,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,468.0,LwQ,144.0,270.0,882.0,GasA,TA,Y,SBrkr,896,0,0,896,0.0,0.0,1,0,2,1,TA,5,Typ,0,NaN,Attchd,1961.0,Unf,1.0,730.0,TA,TA,Y,140,0,0,0,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,6,1958,1958,Hip,CompShg,Wd Sdng,Wd Sdng,BrkFace,108.0,TA,TA,CBlock,TA,TA,No,ALQ,923.0,Unf,0.0,406.0,1329.0,GasA,TA,Y,SBrkr,1329,0,0,1329,0.0,0.0,1,1,3,1,Gd,6,Typ,0,NaN,Attchd,1958.0,Unf,1.0,312.0,TA,TA,Y,393,36,0,0,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,5,5,1997,1998,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,PConc,Gd,TA,No,GLQ,791.0,Unf,0.0,137.0,928.0,GasA,Gd,Y,SBrkr,928,701,0,1629,0.0,0.0,2,1,3,1,TA,6,Typ,1,TA,Attchd,1997.0,Fin,2.0,482.0,TA,TA,Y,212,34,0,0,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,6,6,1998,1998,Gable,CompShg,VinylSd,VinylSd,BrkFace,20.0,TA,TA,PConc,TA,TA,No,GLQ,602.0,Unf,0.0,324.0,926.0,GasA,Ex,Y,SBrkr,926,678,0,1604,0.0,0.0,2,1,3,1,Gd,7,Typ,1,Gd,Attchd,1998.0,Fin,2.0,470.0,TA,TA,Y,360,36,0,0,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,Inside,Gtl,StoneBr,Norm,Norm,TwnhsE,1Story,8,5,1992,1992,Gable,CompShg,HdBoard,HdBoard,None,0.0,Gd,TA,PConc,Gd,TA,No,ALQ,263.0,Unf,0.0,1017.0,1280.0,GasA,Ex,Y,SBrkr,1280,0,0,1280,0.0,0.0,2,0,2,1,Gd,5,Typ,0,NaN,Attchd,1992.0,RFn,2.0,506.0,TA,TA,Y,0,82,0,0,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [12]:
def convert(data):
    
    def change_to_float(x):
        return 0 if x == 'None' else float(x)


    data.drop(columns=['GarageArea', '1stFlrSF', 'TotRmsAbvGrd'], inplace=True)
    
    missing_count = data.isnull().sum().sort_values(ascending=False)
    missing_percentage = missing_count/len(data) * 100
    missing_df = pd.concat([missing_count, missing_percentage], axis=1, keys=['Count', 'Percentage'])
    missing_df = missing_df[missing_df['Count'] > 0]
    for f in missing_df.index:
        if f not in ['MiscFeature', 'LotFrontage', 'MasVnrArea', 'MasVnrType', 'Electrical']:
            data[f] = np.where(data[f].isnull(), 'None', data[f])
            
    data.drop(columns=['MiscFeature', 'LotFrontage', 'MasVnrType', 'MasVnrArea'], inplace=True)
#     data.dropna(how='any', subset=['Electrical'], inplace=True)
    
    features_to_be_normalized = ['GrLivArea', 'TotalBsmtSF', 'YearBuilt', 'YearRemodAdd']
    data['TotalBsmtSF'] = data['TotalBsmtSF'].map(change_to_float)
    
#     skew_type = ['p', 'p', 'n', 'n']
    for f in ['GrLivArea', 'TotalBsmtSF']:
        
#             pdb.set_trace()
#             if skew_type[idx] == 'p':
                data[f] = np.log(1+data[f])
#             else:
#                 data[f] = np.sqrt((data['YrSold'] - data[f]))

                
    cat_features = [f for f in data.columns if data[f].dtype=='O']
    label = LabelEncoder()

    for f in cat_features:
        data[f] = label.fit_transform(data[f])
    
    return data

In [13]:
import pdb

In [14]:
test = pd.read_csv('database/test.csv', index_col=0)
test = convert(test)

In [15]:
test.head()

,MSSubClass,MSZoning,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscVal,MoSold,YrSold,SaleType,SaleCondition
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1461,20,3,11622,1,1,3,3,0,4,0,12,1,2,0,2,5,6,1961,1961,1,0,11,13,3,4,1,4,4,3,5,364,3,21,329,6.783325,0,4,1,3,0,0,6.799056,0,0,1,0,2,1,4,7,0,3,1,46,3,1,4,5,2,140,0,0,0,120,0,2,2,0,6,2010,9,4
1462,20,4,14267,1,1,0,3,0,0,0,12,2,2,0,2,6,6,1958,1958,3,0,12,14,3,4,1,4,4,3,0,634,6,0,438,7.192934,0,4,1,3,0,0,7.192934,0,0,1,1,3,1,2,7,0,3,1,43,3,1,4,5,2,393,36,0,0,0,0,2,4,12500,6,2010,9,4
1463,60,4,13830,1,1,0,3,0,4,0,8,2,2,0,4,5,5,1997,1998,1,0,11,13,3,4,2,2,4,3,2,569,6,0,135,6.834109,0,2,1,3,701,0,7.396335,0,0,2,1,3,1,4,7,1,5,1,82,0,2,4,5,2,212,34,0,0,0,0,2,2,0,3,2010,9,4
1464,60,4,9978,1,1,0,3,0,4,0,8,2,2,0,4,6,6,1998,1998,1,0,11,13,3,4,2,4,4,3,2,455,6,0,371,6.831954,0,0,1,3,678,0,7.380879,0,0,2,1,3,1,2,7,1,2,1,83,0,2,4,5,2,360,36,0,0,0,0,2,4,0,6,2010,9,4
1465,120,4,5005,1,1,0,1,0,4,0,22,2,2,4,2,8,5,1992,1992,1,0,6,6,2,4,2,2,4,3,0,238,6,0,4,7.155396,0,0,1,3,0,0,7.155396,0,0,2,0,2,1,2,7,0,3,1,77,2,2,4,5,2,0,82,0,0,144,0,2,4,0,1,2010,9,4


In [178]:
preds = xgb.predict(test)
preds = np.exp(preds)-1 # Reversing the Log Normalization

In [184]:
def create_submission_file(preds):
    df              = pd.DataFrame()
    df['Id']        = range(1461, 2920)
    df['SalePrice'] = preds

    df.to_csv('database/predictions.csv', index=False)

create_submission_file(preds)

In [31]:
a = np.log(1+500000)
a

13.122365377402328

In [33]:
np.exp(a)-1

499999.99999999977